### 비정형 데이터 융합형 기업 예측 모델
* 뉴스나 전문가 의견및 여론 기반한 비정형 데이터 수집
* 주가 등 정형 데이터 수집
* 기업의 주가 영향 요소 예측등을 통한
* 기업의 위험관리및 투자예측 분석

설립일:1955년 12월 9일 (65주년)[2]
주소
서울특별시 서초구 신반포로 194 (반포동)[3]
업종명
백화점
상장유무
상장기업
기업규모
대기업
상장시장
유가증권시장 (1985년 ~ 현재)
대표이사
차정호
종목코드
004170
자본금
492억 2천만원 (2019)
매출액
1조 5,576억원 (2019)
영업이익
2,220억 9천만원 (2019)
당기순이익
7,059억 5천만원 (2019)
직원 수
2,745명 (2020.06.30)

### 분석요소
* 뉴스 :
* SNS  :
* 댓글 :
* 재무재표: 
* 기업분석 지표:

* 제품 매출 현황

* 시가 :
* 종가 :
* 고가 :
* 저가 :
* 거래량:

### 네이버 뉴스 수집
* 신세계
* url = https://search.naver.com/search.naver?query=%EC%8B%A0%EC%84%B8%EA%B3%84&where=news&ie=utf8&sm=nws_hty

In [32]:
url='https://search.naver.com/search.naver?&where=news&query={0}&sm=tab_pge&sort=1&photo=0&field=0&reporter_article=&pd=0&ds=&de=&docid=&nso=so:dd,p:all,a:all&mynews=1&start={1}&refresh_start=0'.format('아모레퍼시픽',11)
url

'https://search.naver.com/search.naver?&where=news&query=아모레퍼시픽&sm=tab_pge&sort=1&photo=0&field=0&reporter_article=&pd=0&ds=&de=&docid=&nso=so:dd,p:all,a:all&mynews=1&start=11&refresh_start=0'

In [33]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup as bsp
import matplotlib
import matplotlib.pyplot as plt
import sqlite3
import re
from datetime import datetime, timedelta,date
import sys

In [34]:
def calcDate(txt=''):
    #print(txt)
    rdate=''
    nowd=datetime.now()
    pattern='[^\d]'
    num=int(re.sub(pattern,'',txt))
    if('일' in txt):
        res=nowd + timedelta(days=num*-1)
    elif('.' in txt):
        ds=txt.replace('.','-')
        #print(ds)
        res=ds+' 00:00:00.000000'
    else:
        res=nowd + timedelta(hours=num*-1)
    return res

In [35]:
calcDate('2021.2.1')

'2021-2-1 00:00:00.000000'

In [36]:
def makeKey(txt=''):
    txt_=re.sub('http\S*//',' ',txt)
    key=re.sub('[^a-zA-Z0-9%]','',txt_)
    return key

In [37]:
def getData(whr=''):
    dbname='amorenews.db'
    with sqlite3.connect(dbname) as conn:
        cursor=conn.cursor()
        sql='select * from anews '+whr
        res=cursor.execute(sql).fetchall()
    return(res)

In [38]:
def insMain(d):
    dbname='amorenews.db'
    with sqlite3.connect(dbname) as conn:
        cursor=conn.cursor()
        sql='select * from anews where akey="'+d[5]+'"'
        res=cursor.execute(sql).fetchall()
        if(len(res)>0):
            print("이미 존재 합니다.:"+d[1])
        else:
            sql='insert into anews (acate,atitle,acomp,adate,aurl,akey) values(?,?,?,?,?,?)'
            res=cursor.execute(sql,d)
            print('.',end='')
        

In [39]:
def getPage(url):
    res=requests.get(url)
    src=res.content.strip()
    htmls=bsp(src,'html.parser')
    print(url)
    tgts=htmls.select('ul.list_news > li')
    if(len(tgts)==0):
        print('데이터가 없습니다.'+url)
        sys.exit(1)
    for tgt in tgts:
        mainarti=tgt.select('div.news_wrap')
        subartis=tgt.select('ul.list_cluster > li.sub_bx')
        if(len(mainarti)>0):
            title_= mainarti[0].select('a.news_tit')[0].text
            title=re.sub('[^ㄱ-힣0-9 a-zA-Z]',' ',title_)
            comp=mainarti[0].select('.info.press')[0].text.replace('언론사 선정','')
            date_=mainarti[0].select('span.info')[0].text.replace('시간 전','')
            date=calcDate(date_)
            url=mainarti[0].select('a.news_tit')[0]['href']
            key=makeKey(url)
            opt=1
            d=(opt,title,comp,date,url,key)
            insMain(d)
            for subarti in subartis:
                title_= subarti.select('a.sub_tit')[0].text
                title=re.sub('[^ㄱ-힣0-9 a-zA-Z]',' ',title_)
                comp=subarti.select('.sub_txt.press')[0].text.replace('언론사 선정','')
                date_=subarti.select('span.sub_txt')[0].text.replace('시간 전','').replace('네이버뉴스','')
                date=calcDate(date_)
                url=subarti.select('a.sub_tit')[0]['href']
                key=makeKey(url)
                opt=2
                d=(opt,title,comp,date,url,key)
                insMain(d)



In [40]:
urlsrc='https://search.naver.com/search.naver?&where=news&query={0}&start={1}'
for i in range(1,100):
    url=urlsrc.format('아모레퍼시픽',i*10+1)
    print(url)
    getPage(url)

https://search.naver.com/search.naver?&where=news&query=아모레퍼시픽&start=11
https://search.naver.com/search.naver?&where=news&query=아모레퍼시픽&start=11
..........https://search.naver.com/search.naver?&where=news&query=아모레퍼시픽&start=21
https://search.naver.com/search.naver?&where=news&query=아모레퍼시픽&start=21
..........https://search.naver.com/search.naver?&where=news&query=아모레퍼시픽&start=31
https://search.naver.com/search.naver?&where=news&query=아모레퍼시픽&start=31
..........https://search.naver.com/search.naver?&where=news&query=아모레퍼시픽&start=41
https://search.naver.com/search.naver?&where=news&query=아모레퍼시픽&start=41
..........https://search.naver.com/search.naver?&where=news&query=아모레퍼시픽&start=51
https://search.naver.com/search.naver?&where=news&query=아모레퍼시픽&start=51
..........https://search.naver.com/search.naver?&where=news&query=아모레퍼시픽&start=61
https://search.naver.com/search.naver?&where=news&query=아모레퍼시픽&start=61
..........https://search.naver.com/search.naver?&where=news&query=아모레퍼시픽&start=71
http

https://search.naver.com/search.naver?&where=news&query=아모레퍼시픽&start=171
이미 존재 합니다.:아모레퍼시픽  脫플라스틱 앞장 친환경 종이 용기 개발
이미 존재 합니다.:아모레퍼시픽  특허 기술 적용한  친환경 화장품 종이 용기  개발
이미 존재 합니다.:아모레퍼시픽  특허 기술 적용  친환경 화장품 종이 용기  개발
이미 존재 합니다.:아모레퍼시픽  화장품 종이 용기 개발  플라스틱 70  줄여
이미 존재 합니다.:아모레퍼시픽  유통기한 늘린 종이 튜브 개발    친환경 용기 한계 극복
이미 존재 합니다.:아모레퍼시픽  글로벌 RE100  가입 국내 뷰티업계 처음
이미 존재 합니다.:아모레퍼시픽  국내 뷰티업계 최초 글로벌 RE100 가입
이미 존재 합니다.:아모레퍼시픽  국내 뷰티업계 최초  글로벌 RE100  가입
이미 존재 합니다.:아모레퍼시픽  국내외 전 사업장  글로벌 RE100  가입
.이미 존재 합니다.:아모레퍼시픽 바이탈뷰티  브랜드 모델로 배우 송혜교 발탁
이미 존재 합니다.:아모레 바이탈뷰티  모델로 배우 송혜교 발탁
이미 존재 합니다.:아모레퍼시픽 바이탈뷰티  모델 송혜교 발탁
이미 존재 합니다.:아모레퍼시픽 바이탈뷰티  브랜드 모델로 배우 송혜교 발탁
.이미 존재 합니다.:아모레퍼시픽  업계 첫 글로벌 RE100 가입 친환경 경영 인증
이미 존재 합니다.:아모레퍼시픽  국내 뷰티업계 최초 글로벌 RE100 가입
이미 존재 합니다.:아모레퍼시픽  글로벌 RE100  가입
..이미 존재 합니다.:불필요한 플라스틱 줄인다  아모레퍼시픽  친환경 화장품 종이 용기 개발
이미 존재 합니다.:종이로 화장품 용기를  아모레퍼시픽  특허 기술 적용 친환경 용기 개발
이미 존재 합니다.:아모레퍼시픽  특허 기술 적용한 친환경 화장품 종이 용기 개발
이미 존재 합니다.:아모레  종이 화장품용기 개발
이미 존재 합니다.:아모레퍼시픽  특허기술 적용  화장품 종이 용기  개발 플라스틱 사용량 70    


https://search.naver.com/search.naver?&where=news&query=아모레퍼시픽&start=231
이미 존재 합니다.:아모레퍼시픽  국내 뷰티업계 최초 글로벌 RE100 가입
이미 존재 합니다.:아모레퍼시픽  그린 경영   화장품 업계 첫 RE100 가입
이미 존재 합니다.:아모레퍼시픽  글로벌 RE100 가입  전력 100 를 재생에너지로 조달 캠페인
이미 존재 합니다.:아모레퍼시픽  국내 뷰티업계 최초 글로벌 RE100 가입
이미 존재 합니다.: 위드ESG  아모레퍼시픽  ESG 경영 강화로 미래 생각한다
이미 존재 합니다.:아모레퍼시픽  뷰티업계 최초 RE100 가입 전력 100  재생에너지로 조달
.이미 존재 합니다.:아모레퍼시픽  국내 뷰티업계 최초 글로벌 RE100 가입
..이미 존재 합니다.: 예쁜 쓰레기 줄이자  아모레  종이 화장품 용기  개발
이미 존재 합니다.:아모레퍼시픽  플라스틱 70  낮춘 친환경 화장품 종이 용기 개발
이미 존재 합니다.:아모레퍼시픽  친환경 종이 용기 개발 플라스틱 사용량 70  
..이미 존재 합니다.: 아모레퍼시픽   친환경 화장품 종이 용기 개발했다
이미 존재 합니다.:아모레퍼시픽  특허 기술 적용한  친환경 종이 용기  개발
.이미 존재 합니다.:아모레퍼시픽  특허 기술 적용한 친환경 화장품 종이 용기 개발  기존 용기 대   
이미 존재 합니다.:아모레퍼시픽  친환경 화장품 종이 용기 개발     국내 최초 장기간 유통 가능
이미 존재 합니다.:아모레 바이탈뷰티  브랜드 모델로 송혜교 발탁
이미 존재 합니다.:아모레 바이탈뷰티  브랜드 모델로 배우 송혜교 발탁
이미 존재 합니다.:아모레퍼시픽  바이탈뷰티 모델로 배우 송혜교 발탁  변치않는 아름다움 
이미 존재 합니다.:아모레  배우 송혜교  바이탈뷰티 브랜드  모델로 발탁
이미 존재 합니다.:아모레퍼시픽 바이탈뷰티  배우 송혜교 새 모델로 발탁
이미 존재 합니다.:아모레퍼시픽 마몽드   코튼 베일 스킨 베이스  출시
이미 존

https://search.naver.com/search.naver?&where=news&query=아모레퍼시픽&start=291
이미 존재 합니다.:아모레퍼시픽  특허 기술 적용한 친환경 화장품 종이 용기 개발
이미 존재 합니다.:특허 기술 적용한 아모레  친환경 화장품 종이 용기 
이미 존재 합니다.:아모레퍼시픽복지재단  용산 공중화장실  아리따운 화장방  오픈
이미 존재 합니다.:아모레퍼시픽복지재단  안전 위생 갖춘 공중화장실  아리따운 화장방  개장
이미 존재 합니다.:아모레퍼시픽재단  아리따운 화장방  준공
이미 존재 합니다.:아모레퍼시픽복지재단  안전 위생 갖춘 공중화장실  아리따운 화장방  오픈
이미 존재 합니다.:아모레퍼시픽복지재단  공중화장실  아리따운 화장방  오픈
이미 존재 합니다.: 아모레퍼시픽   신재생에너지 캠페인 RE100 가입
이미 존재 합니다.:아모레퍼시픽  국내 뷰티업계 최초 글로벌  RE100  가입   2030년까지 100  목   
이미 존재 합니다.: 전력 100  재생에너지로 조달  아모레퍼시픽   글로벌 RE100  캠페인 동참
이미 존재 합니다.:아모레퍼시픽  국내 뷰티업계 최초  글로벌 RE100  가입
..이미 존재 합니다.:아모레퍼시픽  국내 뷰티업계 최초 글로벌 RE100 가입
이미 존재 합니다.:친환경 경영 실천    아모레퍼시픽  글로벌 RE100 가입
이미 존재 합니다.:아모레퍼시픽  국내 뷰티업계 최초 글로벌 RE100 가입    2030년까지 RE100 목   
이미 존재 합니다.:아모레퍼시픽 국내 전 사업장  글로벌 RE100 가입     국내 뷰티업계로는 최초
이미 존재 합니다.: 기후문제 해결 위해  아모레퍼시픽  국내 최초 글로벌 RE100 가입 글로벌   
이미 존재 합니다.:아모레퍼시픽 일리윤   저자극 이지워시 선크림  출시
이미 존재 합니다.:아모레퍼시픽 일리윤  저자극 이지워시 선크림 출시
이미 존재 합니다.:아모레퍼시픽 일리윤  저자극 이지워시 선크림 출시
이미 존재 합니다.:아모레

https://search.naver.com/search.naver?&where=news&query=아모레퍼시픽&start=351
이미 존재 합니다.:아모레퍼시픽 큐브미  프리미엄 이너뷰티 앰플  퍼펙트큐브 플러스  출시
이미 존재 합니다.:아모레퍼시픽 큐브미  마시는 이너뷰티 앰플  퍼펙트큐브 플러스  출시
이미 존재 합니다.:아모레퍼시픽 일리윤   저자극 이지워시 선크림  선봬
이미 존재 합니다.:아모레퍼시픽  일리윤   저자극 이지워시 선크림 출시
이미 존재 합니다.: 트러블 없이 세안도 쉽게  아모레퍼시픽 일리윤  저자극 이지워시 선크림    
이미 존재 합니다.:아모레퍼시픽  일리윤 저자극 이지워시 선크림 출시
이미 존재 합니다.:아모레퍼시픽 일리윤   쿨링 진정  저자극 이지워시 선크림 출시
이미 존재 합니다.:아모레퍼시픽복지재단  공중화장실  아리따운 화장방  오픈
이미 존재 합니다.:아모레퍼시픽복지재단  안전 위생 갖춘 공중화장실  아리따운 화장방  오픈
이미 존재 합니다.:아모레퍼시픽복지재단 공중화장실 아리따운 화장방 오픈
이미 존재 합니다.:아모레퍼시픽복지재단  안전 위생 갖춘 공중화장실  아리따운 화장방  오픈
이미 존재 합니다.: 뉴스투데이 L  아모레퍼시픽  여성들 위한 공중화장실  아리따운 화장방     
이미 존재 합니다.: SR환경  아모레퍼시픽  글로벌 RE100 가입
이미 존재 합니다.:아모레퍼시픽  국내 뷰티업계 최초  글로벌 RE100  가입
이미 존재 합니다.:서경배 아모레퍼시픽   LG생건에 매출 연봉 반토막
이미 존재 합니다.:연봉도 역전됐다 LG생건 차석용 39억 아모레 서경배 22억
이미 존재 합니다.:아모레퍼시픽 프리메라   바하 버블 필링 클렌저  출시
이미 존재 합니다.:아모레퍼시픽 프리메라   바하 BHA  버블 필링 클렌저  선봬
이미 존재 합니다.:아모레퍼시픽 프리메라   바하 BHA  버블 필링 클렌저  출시
이미 존재 합니다.:아모레퍼시픽  프리메라    바하 BHA  버블 필링 클렌저  선보여
이미 

...

https://search.naver.com/search.naver?&where=news&query=아모레퍼시픽&start=601
이미 존재 합니다.:아모레퍼시픽  GS칼텍스와 플라스틱 공병 재활용 MOU
이미 존재 합니다.:아모레퍼시픽 GS칼텍스  플라스틱 공병 재활용 업무 협약
이미 존재 합니다.:아모레퍼시픽  GS칼텍스와 플라스틱 공병 재활용 위해  맞손 
이미 존재 합니다.:아모레퍼시픽  GS칼텍스와 공병 재활용 업무 협약 체결
.이미 존재 합니다.:아모레퍼시픽  AI가 립스틱 색 추천 즉석 제조
이미 존재 합니다.:아모레퍼시픽  CES 2021 참가  혁신상 기술 2건 전시
이미 존재 합니다.:아모레퍼시픽   CES 2021  참가 맞춤형 뷰티 기술 소개
이미 존재 합니다.:아모레퍼시픽  CES 2021 참가
이미 존재 합니다.:아모레퍼시픽  CES 혁신상 기술 전 세계 선보인다
이미 존재 합니다.:아모레퍼시픽  퍼시픽글라스 지분 매각한다
이미 존재 합니다.:아모레퍼시픽그룹  화장품 공병 만드는 자회사 퍼시픽글라스 매각
이미 존재 합니다.:아모레퍼시픽그룹   퍼시픽글라스  지분 60  佛 베르상스에 매각
이미 존재 합니다.:아모레그룹  佛  베르상스 에 자회사 퍼시픽글라스 지분 60  매각
이미 존재 합니다.:아모레퍼시픽   희망가게  창업주 공개 모집
이미 존재 합니다.:아모레퍼시픽   희망가게  창업주 공개 모집
이미 존재 합니다.:아모레퍼시픽   희망가게  창업주 공개 모집
이미 존재 합니다.:아모레퍼시픽   희망가게  창업주 공개 모집 한부모 여성 자립 지원
.이미 존재 합니다.:아모레퍼시픽미술관  현대미술 소장품 특별전 개최
이미 존재 합니다.: 차별화된 쇼핑 경험 제공  아모레퍼시픽 이베이코리아  업무제휴 협약
이미 존재 합니다.:아모레퍼시픽  이베이코리아와 업무 제휴 협약 JBP  체결
이미 존재 합니다.:아모레퍼시픽  이베이코리아와 전략적 협업 강화
.이미 존재 합니다.:아모레퍼시픽   오프화이트 와 맞손  프로텍션 박스 선봬 
...이

https://search.naver.com/search.naver?&where=news&query=아모레퍼시픽&start=691
이미 존재 합니다.: 협업의 제왕  버질 아블로  이번엔 아모레퍼시픽과  윷놀이  냈다
이미 존재 합니다.:아모레퍼시픽X오프화이트  한정판 제품 출시
이미 존재 합니다.:아모레퍼시픽  한국 기업 최초로  오프화이트  협업상품 선봬
이미 존재 합니다.:아모레퍼시픽  오프화이트와 맞손  한정판 제품 출시
이미 존재 합니다.:아모레퍼시픽X오프화이트 한정판 제품 출시
이미 존재 합니다.:아모레퍼시픽  아름다운재단과  희망가게  창업주 모집
....이미 존재 합니다.:아모레퍼시픽   헤어살롱 코디네이터  양성
이미 존재 합니다.:아모레퍼시픽  취약계층 여성 위해 비대면 교육 실시
.이미 존재 합니다.:아모레퍼시픽 이베이코리아와 업무 제휴 협약 JBP  체결
.이미 존재 합니다.:아모레퍼시픽  4R 전략  친환경 동참
이미 존재 합니다.: 환경을 생각합니다  아모레퍼시픽  설 선물세트  지구를 부탁해  선봬
이미 존재 합니다.:아모레퍼시픽  화장품 플라스틱 문제 해결에  앞장 
이미 존재 합니다.:아모레퍼시픽  화장품 업계  플라스틱 줄이기  앞장
.이미 존재 합니다.:아모레퍼시픽표  맞춤형 뷰티  CES 2021 혁신상 수상
이미 존재 합니다.:아모레퍼시픽  CES 2021 혁신상 수상 기술 2건 전시
이미 존재 합니다.:아모레퍼시픽  CES 2021서 AI 활용한 뷰티기술 공개
이미 존재 합니다.:아모레퍼시픽  CES 2021 혁신상 수상 기술 2건 전시
..이미 존재 합니다.:아모레퍼시픽  취약계층 여성 지원에 10억 쾌척
.이미 존재 합니다.:K 뷰티 선도자에서 추격자로 아모레퍼시픽  지난해 영업이익 70  줄었다
이미 존재 합니다.: 이슈분석  아모레퍼시픽의 돌파구 전략은 무엇
.https://search.naver.com/search.naver?&where=news&query=아모레퍼시픽&start=701
https://search

https://search.naver.com/search.naver?&where=news&query=아모레퍼시픽&start=781
이미 존재 합니다.:아모레퍼시픽X오프화이트 한정판 제품 출시
이미 존재 합니다.:아모레퍼시픽 오프화이트  한정판 제품 출시
이미 존재 합니다.:아모레퍼시픽X오프화이트 한정판 출시
이미 존재 합니다.:아모레  名品 오프화이트 협업 중국 화장품 시장 반격 나선다
.이미 존재 합니다.:아모레퍼시픽   오프화이트  한정판 제품 출시
이미 존재 합니다.:아모레퍼시픽  오프화이트와 협업 제품 출시  한국 최초
...이미 존재 합니다.: 포토  아모레퍼시픽미술관 현대미술 소장품 특별전
이미 존재 합니다.:아모레퍼시픽  현대미술 소장품 특별전 개최
이미 존재 합니다.:아모레퍼시픽미술관  현대미술 소장품 특별전
이미 존재 합니다.:아모레퍼시픽미술관  현대미술 소장품 특별전  APMA  CHAPTER THREE 
.이미 존재 합니다.:아모레퍼시픽  CES 혁신상  맞춤형 뷰티기술 선봬
이미 존재 합니다.:아모레퍼시픽  환경부 통합환경허가 취득 화장품 업계 최초
이미 존재 합니다.:아모레퍼시픽  업계 최초 환경부 통합환경허가 취득
.이미 존재 합니다.:아모레퍼시픽  이베이코리아와 전략적 협업 강화
이미 존재 합니다.:이베이코리아 아모레퍼시픽  전략적 협업 강화
이미 존재 합니다.:아모레퍼시픽  사회복지공동모금회에 10억원 기부
이미 존재 합니다.:아모레퍼시픽  사회복지공동모금회에 10억 원 기부
이미 존재 합니다.:아모레퍼시픽  사회복지공동모금회에 10억 원 기부
이미 존재 합니다.:아모레퍼시픽  사회복지공동모금회에 10억원 기부
.이미 존재 합니다.:아모레퍼시픽  한부모여성  희망가게  창업주 모집
이미 존재 합니다.:아모레퍼시픽   희망가게  창업주 공모   최대 4천만원 제공
이미 존재 합니다.: 여성 경제적 자립 지원     아모레퍼시픽   희망가게  창업주 공개 모집
..이미 존재 합니다.:아모레퍼시픽  확진자 발생으로 본사 건물 폐쇄    

https://search.naver.com/search.naver?&where=news&query=아모레퍼시픽&start=871
....이미 존재 합니다.:아모레퍼시픽  CES 2021 혁신상 수상 기술 2건 전시
이미 존재 합니다.:아모레퍼시픽  CES 2021 혁신상 수상 기술 2건 전시
....이미 존재 합니다.:아모레퍼시픽미술관  현대미술 소장품 특별전 개최
이미 존재 합니다.:아모레퍼시픽미술관  현대미술 소장품 특별전 개최
.이미 존재 합니다.:아모레퍼시픽미술관  현대미술 소장품 특별전  APMA  챕터3  개최
.이미 존재 합니다.:아모레퍼시픽  환경부 통합환경허가 취득
이미 존재 합니다.:아모레퍼시픽  환경부 통합환경허가 취득   선진 환경관리
.이미 존재 합니다.: 기능도 디자인도 꽉  아모레퍼시픽 LG생활건강  프로 필링 트리플 AHA 클   
이미 존재 합니다.:아모레 아이오페   프로 필링 트리플 AHA 클렌저  출시
이미 존재 합니다.:아모레퍼시픽   아이오페 더 비타민 C23  특허기술상 수상
이미 존재 합니다.:아모레퍼시픽   아이오페 더 비타민 C23  특허기술상 수상
이미 존재 합니다.:아모레퍼시픽   아이오페 더 비타민 C23  특허기술상 수상
이미 존재 합니다.:아모레퍼시픽  아이오페 더 비타민 C23  특허기술상 수상
.이미 존재 합니다.:아모레 바이탈뷰티   녹차에서 온 유산균 포스트  선봬
.이미 존재 합니다.:아모레퍼시픽복지재단  여성 이용시설 공간 리모델링 지원 공모 시작
이미 존재 합니다.:아모레퍼시픽복지재단  여성 이용시설 리모델링 지원 접수
이미 존재 합니다.:아모레퍼시픽복지재단   2021년 공간문화개선사업  공모 접수
.https://search.naver.com/search.naver?&where=news&query=아모레퍼시픽&start=881
https://search.naver.com/search.naver?&where=news&query=아모레퍼시픽&start=881
이미 존재 합니다.:아모레퍼시픽 

https://search.naver.com/search.naver?&where=news&query=아모레퍼시픽&start=961
이미 존재 합니다.:아모레퍼시픽  희망가게 창업주 공개 모집
이미 존재 합니다.:아모레퍼시픽   희망가게  창업주 공개 모집
이미 존재 합니다.:아모레퍼시픽  한부모 여성의 경제적 자립 지원  희망가게  창업주 공개 모집
이미 존재 합니다.:아모레퍼시픽그룹  자회사 퍼시픽글라스 지분 매각
이미 존재 합니다.:아모레퍼시픽그룹   퍼시픽글라스  지분 60  佛 기업에 판다
이미 존재 합니다.: 자회사 팔았다  아모레퍼시픽그룹  퍼시픽글라스 지분 매각
.이미 존재 합니다.:아모레퍼시픽   GS칼텍스  플리스틱 공병 재활용 업무협약 체결
이미 존재 합니다.:GS칼텍스 아모레퍼시픽  플라스틱 공병 재활용 사업 추진
...이미 존재 합니다.:아모레퍼시픽  맞춤형 립 토너 패드 제조 기술 CES 전시
이미 존재 합니다.:아모레퍼시픽  CES 2021 혁신상 수상 기술 2건 전시
이미 존재 합니다.:아모레퍼시픽  CES 2021 혁신상 수상 기술 2건 전시
이미 존재 합니다.:아모레퍼시픽  CES 2021 참가   혁신상 기술 2건 전시
.이미 존재 합니다.:아모레퍼시픽   CES 2021  참가 혁신상 기술 2건 전시
이미 존재 합니다.: CES 2021  아모레퍼시픽  맞춤형 혁신 뷰티기술 선봬
이미 존재 합니다.:아모레퍼시픽  CES 2021 참가 혁신상 수상 기술 2건 전시
이미 존재 합니다.:아모레퍼시픽  CES2021에서 고객 맞춤형 뷰티기술 2종 선보여
....이미 존재 합니다.: 브리핑  이랜드 LF 아모레퍼시픽 코오롱FnC
이미 존재 합니다.:아모레퍼시픽미술관  현대미술 소장품 특별전  APMA  CHAPTER THREE  개최
이미 존재 합니다.:아모레퍼시픽미술관   APMA  CHAPTER THREE  특별전
이미 존재 합니다.:아모레퍼시픽미술관  현대미술 소장품 특별전 개최
.https://search.naver.com/se